## Plotting Installation:

The plotting package allows you to make an interactive CRN plot. Plotting requires the [Bokeh](https://docs.bokeh.org/en/latest/docs/installation.html) and [ForceAtlas2](https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0098679) libraries to be installed on your machine. Bokeh is used for plotting, ForceAtlas2 is used for graph layout. To install, type the following into the consol:

    Conda install bokeh
    pip install fa2

Alternatively, follow the installation instructions in the links above.




## Plotting Example
The CRN plot has a default representation:
* species are circles
    - orange circles are RNA species
    - blue circles are complexes
    - green circles are proteins
    - grey circles are DNA
    -there's always a purple circle that represents nothing. If your CRN doesn't have reactions that go to nothing, then it won't have any connections.
    
* reactions are squares

Click on a node (either reaction or species) and all arrows including that node will be highlighted
Mouse over a node (either reaction or species) and a tooltip will tell you what it is.

### Create a BioCRNpyler Model
Here we model a DNA assembly with a regulated promoter in a TxTl mixture.

In [2]:
from biocrnpyler import *


txtl = TxTlExtract("mixture1", parameter_file = 'default_parameters.txt')
dna = DNAassembly("mydna",promoter=RegulatedPromoter("plac",["laci"]),rbs="UTR1",protein="GFP", initial_concentration = 10)
txtl.add_component(dna)
crn1 = txtl.compile_crn()
crn1.add_reactions([Reaction.from_massaction([Species("mydna",material_type="rna")],[],k_forward=0.1)])
print(crn1.pretty_print())

Species(N = 13) = {
dna[mydna] (@ 10),  
   found_key=(mech=initial concentration, partid=None, name=mydna).
   search_key=(mech=initial concentration, partid=mixture1, name=mydna).
complex[protein[Ribo]:rna[mydna]] (@ 0),  complex[protein[RNAase]:rna[mydna]] (@ 0),  rna[mydna] (@ 0),  protein[laci] (@ 0),  complex[dna[mydna]:2x_protein[laci]] (@ 0),  complex[dna[mydna]:protein[RNAP]] (@ 0),  complex[complex[protein[Ribo]:rna[mydna]]:protein[RNAase]] (@ 0),  complex[complex[dna[mydna]:2x_protein[laci]]:protein[RNAP]] (@ 0),  protein[Ribo] (@ 0),  protein[RNAase] (@ 0),  protein[RNAP] (@ 0),  protein[GFP] (@ 0),  
}

Reactions (12) = [
0. dna[mydna]+protein[RNAP] <--> complex[dna[mydna]:protein[RNAP]]
 Kf=k_forward * dna_mydna * protein_RNAP
 Kr=k_reverse * complex_dna_mydna_protein_RNAP_
  k_forward=100.0
  found_key=(mech=None, partid=None, name=kb).
  search_key=(mech=transcription_mm, partid=plac_leak, name=kb).
  k_reverse=10.0
  found_key=(mech=None, partid=None, name=ku).
  searc

## Plotting a reaction graph
First we import bokeh in order to plot an interactive graph then use the graphPlot function and the BioCRNpyler.plotting.generate_networkx_graph(crn) function to produce a graph. Mouseover the circles to see which species they identify and the squares to see reactions.

In [3]:
try:
    from bokeh.models import (Plot , Range1d)
    import bokeh.plotting
    import bokeh.io
    bokeh.io.output_notebook() #this makes the graph appear in line with the notebook
    plot = render_network_bokeh(crn1)
    bokeh.io.show(plot) #if you don't type this the plot won't show
except ModuleNotFoundError:
    print('please install the plotting libraries: pip install biocrnpyler[all]')


#mouse over nodes to get a tooltip telling you what they are
#mouse over single lines to outline them
#click on nodes to highlight all edges that touch them
#use the magnifying glass symbol to zoom in and out
#click and drag to move around

#NOTE: this function is not deterministic in how it displays the network, because it uses randomness to push the nodes around

Loading BokehJS ...

D:\anaconda3\lib\site-packages\bokeh\plotting\graph.py:120: UserWarning: Node keys in 'layout_function' don't match node keys in the graph. These nodes may not be displayed correctly.
  warn("Node keys in 'layout_function' don't match node keys in the graph. "


### Advanced options for generate_networkx_graph
The following options are useful changing how plotting works.

To get better control over the way reactions and species text is display using the following keywords (default values shown below):

    use_pretty_print=False
    pp_show_rates=True
    pp_show_attributes=True
    pp_show_material=True

To get better control over the colors of different nodes, use the keywords to set a species' color.
The higher keywords will take precedence.

    repr(species): "color"
    species.name: "color"
    (species.material_type, tuple(specie.attributes)): "color"
    species.material_type: "color"
    tuple(species.attributes): "color"


In [4]:
try:
    from bokeh.models import (Plot , Range1d)
    import bokeh.plotting
    import bokeh.io
    bokeh.io.output_notebook() #this makes the graph appear in line with the notebook

    #this demonstrates the "circle" layout. reactions are in the middle with species on the outside.
    #also, the pretty_print text display style

    colordict = {
        "G1":"red", #will only effect the species dna_G1 and rna_G1
        "protein_GFP": "lightgreen", #will only effect the species protein_GFP
        "protein": "blue" #All protein species, protein_Ribo, protein_RNAase, and protein_RNAP will be blue
        #All other species will be grey by default. This will include all complexes.
    }
    plot = render_network_bokeh(crn1,
                            colordict = colordict,
                            use_pretty_print=True, #uses pretty print
                            pp_show_rates=False, #this would put the reaction rates in the reaction name. It's already listed seperately in the tool tip
                            pp_show_attributes=False,
                            pp_show_material = True, #this lists the material of the species being displayed
                            layout="circle",
    )
    bokeh.io.show(plot)
except ModuleNotFoundError:
    print('please install the plotting libraries: pip install biocrnpyler[all]')


Loading BokehJS ...

In [6]:
#this is an example of how to use the new image plotting. It is designed to combine dnaplotlib and the network representation to show constructs and where proteins are bound to them.

#to use this to its full potential you must make dna_constructs:
txtl = TxTlExtract("mixture1", parameter_file = 'default_parameters.txt')

ptet = RegulatedPromoter("ptet",["tetR"])
utr1 = RBS("utr1")
gfp = CDS("GFP")
t16 = Terminator("t16")

construct = DNA_construct([ptet,utr1,gfp,t16], initial_concentration=5)


dna = DNAassembly("mydna",promoter=RegulatedPromoter("plac",["laci"]),rbs=utr1,protein="GFP", initial_concentration = 10)
txtl.add_component(dna)
txtl.add_component(construct)
crn1 = txtl.compile_crn()
crn1.add_reactions([Reaction.from_massaction([Species("mydna",material_type="rna")],[],k_forward=0.1)])
try:
    #dnaplotlib is a cool library for plotting DNAs.
    #please use my fork located at https://github.com/dr3y/dnaplotlib
    #to install it type: pip install git+git://github.com/dr3y/dnaplotlib.git@master
    import dnaplotlib as dpl
    %matplotlib inline
    dpl_enabled = True
except (ModuleNotFoundError,ImportError) as e:
    dpl_enabled = False

try:
    from bokeh.models import (Plot , Range1d)
    import bokeh.plotting
    import bokeh.io
    bokeh.io.output_notebook() #this makes the graph appear in line with the notebook

    #this demonstrates the "circle" layout. reactions are in the middle with species on the outside.
    #also, the pretty_print text display style

    colordict = {
        "G1":"red", #will only effect the species dna_G1 and rna_G1
        "protein_GFP": "green", #will only effect the species protein_GFP
        "GFP":"lightgreen", #this will affect how the dnaplotlib is drawn
        "protein": "blue" #All protein species, protein_Ribo, protein_RNAase, and protein_RNAP will be blue
        #All other species will be grey by default. This will include all complexes.
    }
    colordict = {}
    if(dpl_enabled):
        images =render_mixture(txtl,crn1,colordict)
    else:
        images = None
    plot = render_network_bokeh(crn1,
                                                colordict = colordict,
                                                use_pretty_print=True, #uses pretty print
                                                pp_show_rates=False, #this would put the reaction rates in the reaction name. It's already listed seperately in the tool tip
                                                pp_show_attributes=False,
                                                pp_show_material = True, #this lists the material of the species being displayed
                                                imagedict =images, #this is a dictionary of which images to show for each species
                                                layout="circle",
    )
    bokeh.io.show(plot)
except ModuleNotFoundError:
    print('please install the plotting libraries: pip install biocrnpyler[all]')

d:\documents\github\biocrnpyler\biocrnpyler\parameter.py:502: UserWarning: parameter file contains no unit column! Please add a column named ['unit', 'units'].
  warn(f"parameter file contains no {accepted_name} column! Please add a "


Loading BokehJS ...

d:\documents\github\biocrnpyler\biocrnpyler\plotting.py:500: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout(pad=0.0001)
0
0
0
0
d:\documents\github\biocrnpyler\biocrnpyler\plotting.py:498: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig = plt.figure(figsize=figsize)
D:\anaconda3\lib\site-packages\bokeh\plotting\graph.py:120: UserWarning: Node keys in 'layout_function' don't match node keys in the graph. These nodes may not be displayed correctly.
  warn("Node keys in 'layout_function' don't match node keys in the graph. "
